In [2]:
import cv2
import numpy as np
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
#movie = 'Control'
#movie = '130721'
#movie ='140721'
#movie ='150721'
#movie ='100621'
movie = '130721_CCA60_RAW'
MOVIE_PATH = DATA_PATH +  movie + '/'
GRAPH_PATH = 'C:/Users/amityu/Gel_Sheet_Graph/'


In [3]:
imgs_float = np.load(MOVIE_PATH + 'np/gel.npy')[:,:,:,256] # replace with your images
imgs = np.zeros(imgs_float.shape, dtype='uint8')
imgs_float = np.nan_to_num(imgs_float)
for t in range(len(imgs_float)):
    img = imgs_float[t]
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    img = img.astype(np.uint8)
    imgs[t] = img

#replace nan with zero

In [6]:
import cv2
import numpy as np
y = 0
# Global variables
ix,iy, iz = -1, 0, -1
y_gap = 20
drawing = False
img_index = 0
r_size = 40 # Define your rectangle size here
coordinates = []

# Define the draw_rectangle callback function
def draw_rectangle(event, x, z, flags, param):
    global ix, iz, drawing, img, img_index, r_size, img_orig

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iz = x, z
        img = img_orig.copy()  # create a copy of the image

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            img_temp = img.copy()
            cv2.rectangle(img_temp, pt1=(ix, iz), pt2=(ix+r_size, iz+r_size), color=(0, 255, 0), thickness=2)
            cv2.imshow('image', img_temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, pt1=(ix, iz), pt2=(ix+r_size, iz+r_size), color= (255,255,255), thickness=2)
        coordinates.append((img_index, ix, iz))
        print(f'Saved coordinates: {coordinates[-1]}')

    elif event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:  # scroll up
            img_index = (img_index + 1) % len(imgs)  # cycle through images
        else:  # scroll down
            img_index = (img_index - 1) % len(imgs)  # cycle through images
        img_orig = imgs[img_index].copy()  # load the new image
        img = img_orig.copy()
        cv2.imshow('image', img)
    data_string = f'Time: {img_index}, X: {x}, Y: {y}, Z: {z}'
    data_img = np.zeros((500,500))
    position = (10, 50)  # bottom-left corner of the text
    fontScale = 0.7
    font = cv2.FONT_HERSHEY_SIMPLEX
    color = (255, 255, 255)  # white color
    thickness = 2  # line thickness
    cv2.putText(data_img, data_string,  position, font, fontScale, color, thickness)
    cv2.imshow('Data', data_img)

# 3D array of images
img = imgs[img_index].copy()

# Create the named window and set the mouse callback
cv2.namedWindow('image')
cv2.namedWindow('Data')
cv2.setMouseCallback('image', draw_rectangle)
#cv2.setMouseCallback('Data', getData)


# Show the image and wait for the user to close the window
while True:
    cv2.imshow('image', img)
    key = cv2.waitKey(1) & 0xFF
    if key  == ord('q'):
        break
    elif key == ord('8'):
        img_index +=1
        if img_index >= len(imgs):
            img_index = 0
        draw_rectangle(None, 0, 0, 1, 0)
    elif key == ord('2'):
        img_index -=1
        if img_index < 0:
            img_index = len(imgs)-1
        draw_rectangle(None, 0, 0, -1, 0)


cv2.destroyAllWindows()

# Print the saved coordinates
print('All saved coordinates:')
for idx, x, y in coordinates:
    print(f'Image index: {idx}, X: {x}, Y: {y}')


All saved coordinates:
